## Just trying out new concept which i learned recently....

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import os
import glob
import tensorflow as tf
from tensorflow import keras

In [ ]:
train = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
train.head()

In [ ]:
train = train.sort_values(by='Id', ascending=True)
test = test.sort_values(by='Id', ascending=True)

train_images = glob.glob("/kaggle/input/petfinder-pawpularity-score/train/*.jpg")
test_images = glob.glob("/kaggle/input/petfinder-pawpularity-score/test/*.jpg")

train['Image'] = train_images
test['Image'] = test_images

In [ ]:
train_image_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
train_dataset = train_image_generator.flow_from_dataframe(dataframe=train, 
                                                        directory='../input/petfinder-pawpularity-score/test',
                                                        x_col="Image",
                                                        y_col="Pawpularity",
                                                        subset="training",
                                                        target_size=(256,256),
                                                        shuffle=True,
                                                        class_mode="raw")


test_dataset = test_image_generator.flow_from_dataframe(dataframe=test,
                                                directory='/kaggle/input/petfinder-pawpularity-score/test',
                                                x_col="Image",
                                                y_col=None,
                                                target_size=(256,256),
                                                shuffle=False,
                                                class_mode=None)

In [ ]:
# building a CNN architecure

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), input_shape=(256, 256, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(filters=40, kernel_size=(3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(40, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=tf.keras.metrics.RootMeanSquaredError())

In [ ]:
model.fit(x=train_dataset, epochs=15)

In [ ]:
model.evaluate(test_dataset)

In [ ]:
predictions = model.predict(test_dataset)

In [ ]:
predictions = predictions.ravel()

In [ ]:
sample_sub = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")
sample_sub.head()

In [ ]:
sub = pd.DataFrame({'Id':test['Id'], 'Pawpularity':predictions})

In [ ]:
sub.to_csv("submission.csv", index = False)